use shingles (bigrams or pairs) to help bump the score. They need to be created at index time however

In [1]:
curl -XDELETE 'localhost:9200/my_index?pretty'

{
  "acknowledged" : true
}


use unigrams for regular search, and bigrams to boots the results

In [2]:
curl -XPUT 'localhost:9200/my_index?pretty' -H 'Content-Type: application/json' -d'
{
    "settings": {
        "number_of_shards": 1,  
        "analysis": {
            "filter": {
                "my_shingle_filter": {
                    "type":             "shingle",
                    "min_shingle_size": 2, 
                    "max_shingle_size": 2, 
                    "output_unigrams":  false   
                }
            },
            "analyzer": {
                "my_shingle_analyzer": {
                    "type":             "custom",
                    "tokenizer":        "standard",
                    "filter": [
                        "lowercase",
                        "my_shingle_filter" 
                    ]
                }
            }
        }
    }
}
'

{
  "acknowledged" : true,
  "shards_acknowledged" : true
}


check to see if our analyzer is working as expected

In [6]:
curl -XGET 'localhost:9200/my_index/_analyze?analyzer=my_shingle_analyzer&pretty' -H 'Content-Type: application/json' -d'
Sue ate the alligator
'

{
  "tokens" : [
    {
      "token" : "sue ate",
      "start_offset" : 1,
      "end_offset" : 8,
      "type" : "shingle",
      "position" : 0
    },
    {
      "token" : "ate the",
      "start_offset" : 5,
      "end_offset" : 12,
      "type" : "shingle",
      "position" : 1
    },
    {
      "token" : "the alligator",
      "start_offset" : 9,
      "end_offset" : 22,
      "type" : "shingle",
      "position" : 2
    }
  ]
}


use the new analyzer

In [7]:
curl -XPUT 'localhost:9200/my_index/_mapping/my_type?pretty' -H 'Content-Type: application/json' -d'
{
    "my_type": {
        "properties": {
            "title": {
                "type": "string",
                "fields": {
                    "shingles": {
                        "type":     "string",
                        "analyzer": "my_shingle_analyzer"
                    }
                }
            }
        }
    }
}
'

{
  "acknowledged" : true
}


add some test data

In [8]:
curl -XPOST 'localhost:9200/my_index/my_type/_bulk?pretty' -H 'Content-Type: application/json' -d'
{ "index": { "_id": 1 }}
{ "title": "Sue ate the alligator" }
{ "index": { "_id": 2 }}
{ "title": "The alligator ate Sue" }
{ "index": { "_id": 3 }}
{ "title": "Sue never goes anywhere without her alligator skin purse" }
'

{
  "took" : 96,
  "errors" : false,
  "items" : [
    {
      "index" : {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "1",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 2,
          "failed" : 0
        },
        "created" : true,
        "status" : 201
      }
    },
    {
      "index" : {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "2",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 2,
          "failed" : 0
        },
        "created" : true,
        "status" : 201
      }
    },
    {
      "index" : {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "3",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 2,
          "failed" : 0
        },
        "created" : tru

see how a regular match query returns

In [9]:
curl -XGET 'localhost:9200/my_index/my_type/_search?pretty' -H 'Content-Type: application/json' -d'
{
   "query": {
        "match": {
           "title": "the hungry alligator ate sue"
        }
   }
}
'

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 3,
    "max_score" : 1.3721708,
    "hits" : [
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "1",
        "_score" : 1.3721708,
        "_source" : {
          "title" : "Sue ate the alligator"
        }
      },
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "2",
        "_score" : 1.3721708,
        "_source" : {
          "title" : "The alligator ate Sue"
        }
      },
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "3",
        "_score" : 0.20077486,
        "_source" : {
          "title" : "Sue never goes anywhere without her alligator skin purse"
        }
      }
    ]
  }
}


now lets boost with the shingles query

In [14]:
curl -XGET 'localhost:9200/my_index/my_type/_search?pretty' -H 'Content-Type: application/json' -d'
{
   "query": {
      "bool": {
         "must": {
            "match": {
               "title": "the hungry alligator ate sue"
            }
         },
         "should": {
            "match": {
               "title.shingles": "the hungry alligator ate sue"
            }
         }
      }
   }
}
'

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 3,
    "max_score" : 1.3721708,
    "hits" : [
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "1",
        "_score" : 1.3721708,
        "_source" : {
          "title" : "Sue ate the alligator"
        }
      },
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "2",
        "_score" : 1.3721708,
        "_source" : {
          "title" : "The alligator ate Sue"
        }
      },
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "3",
        "_score" : 0.20077486,
        "_source" : {
          "title" : "Sue never goes anywhere without her alligator skin purse"
        }
      }
    ]
  }
}


weird, doesn't work as expected :)